In [1]:
using BenchmarkTools

In [11]:
# Calculate distance in meters between two points (ϕ1, λ1) and (ϕ2, λ2)
# on the Earth's surface using the haversine formula. ϕ denotes the latitude
# while λ denotes the longitude.
# See: http://www.movable-type.co.uk/scripts/latlong.html
function haversine_distance(ϕ1, λ1, ϕ2, λ2, R)
    Δϕ = ϕ2 - ϕ1
    Δλ = λ2 - λ1
    a = sinpi(Δϕ / 360.0f0)^2 + cospi(ϕ1 / 180.0f0) * cospi(ϕ2 / 180.0f0) * sinpi(Δλ / 360.0f0)^2
    # c = 2.0f0 * atan(√a, √(1-a))
    c = 2.0f0 * asin(min(1.0f0, √a))
    R*c
end

function haversine_dista32(lat1::Float32, lon1::Float32, lat2::Float32, lon2::Float32, radius::Float32)
    c1 = cospi(lat1 / 180.0f0)
    c2 = cospi(lat2 / 180.0f0)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d1 = sinpi(dlat / 360.0f0)
    d2 = sinpi(dlon / 360.0f0)
    t = d2 * d2 * c1 * c2
    a = d1 * d1 + t
    c = 2.0f0 * asin(min(1.0f0, sqrt(a)))
    return radius * c
end

function haversine_cpu(ϕ1, λ1, ϕ2, λ2, radius)
    c1 = cospi(ϕ1 / 180.0f0)
    c2 = cospi(ϕ2 / 180.0f0)
    Δλ = λ2 - λ1
    Δϕ = ϕ2 - ϕ1
    sΔλ = sinpi(Δλ / 360.0f0)
    sΔϕ = sinpi(Δϕ / 360.0f0)
    t = sΔλ * sΔλ * c1 * c2
    a = sΔϕ * sΔϕ + t
    c = 2.0f0 * asin(min(1.0f0, √a))
    return radius * c
end

haversine_cpu (generic function with 1 method)

In [12]:
R64, R32 = 6371.228e3, 6371.228f3 # average radius of the earth [m]
# @show haversine_distance(10f0, 20f0, 30f0, 40f0, R32)
# @show haversine_cpu(10f0, 20f0, 30f0, 40f0, R32)
haversine_distance(10f0, 20f0, 30f0, 40f0, R32) ≈ haversine_cpu(10f0, 20f0, 30f0, 40f0, R32) ≈ haversine_32(10f0, 20f0, 30f0, 40f0, R32)

true

In [8]:
@benchmark haversine_distance(10, 20, 30, 40, R64)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     107.326 ns (0.00% GC)
  median time:      113.172 ns (0.00% GC)
  mean time:        133.622 ns (4.29% GC)
  maximum time:     42.289 μs (99.72% GC)
  --------------
  samples:          10000
  evals/sample:     946

In [9]:
@benchmark haversine_distance(10f0, 20f0, 30f0, 40f0, R32)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     78.451 ns (0.00% GC)
  median time:      83.655 ns (0.00% GC)
  mean time:        102.412 ns (5.12% GC)
  maximum time:     38.421 μs (99.78% GC)
  --------------
  samples:          10000
  evals/sample:     987

In [13]:
@benchmark haversine_32(10f0, 20f0, 30f0, 40f0, R32)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     58.469 ns (0.00% GC)
  median time:      59.260 ns (0.00% GC)
  mean time:        67.816 ns (0.00% GC)
  maximum time:     194.765 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [14]:
@benchmark haversine_cpu(10f0, 20f0, 30f0, 40f0, R32)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     77.241 ns (0.00% GC)
  median time:      80.059 ns (0.00% GC)
  mean time:        97.582 ns (5.90% GC)
  maximum time:     43.416 μs (99.79% GC)
  --------------
  samples:          10000
  evals/sample:     982